In [120]:
import numpy as np
import matplotlib.pyplot as plt
import time

#initialize matrices
def matrix_init(n, test):
    #dimensions of matrices n
    if test == 1: #matrix from book
        A = np.array([[-2, -3], [6, 7]], dtype=np.float32)
    if test == 2: #A = 10I + B + B^T + B^TB
        B = np.random.normal(size = (n, n))
        Bt = np.transpose(B)
        A = 10 * np.eye(n) + B + Bt + np.matmul(Bt, B)
    return A

#back substitution of upper triangular matrix
def Backsolve(A, b, n): #based on algorithm 6.1
    #coefficient of unknowns matrix A, constants matrix b
    x = np.zeros((n,1))
    for i in range(n - 1, -1, -1):
        x_i = b[i]
        for j in range(i + 1, n):
            x_i = x_i - A[i][j] * x[j]
        x[i] = x_i/A[i][i]
    return x

#compute LU factorization of matrix
def LU_factorize(A, n): #based on algorithm 6.4
    #coefficient of unknowns matrix A
    L = np.eye(n, dtype=np.float32) #set L_ii to 1 (identity matrix)
    U = np.zeros((n, n), dtype=np.float32) #initialize U
    for i in range(n):
        for j in range(i, n):
            u_ij = A[i][j] - np.dot(L[i,:], U[:,j])
            l_ji = A[j][i] - np.dot(L[j,:], U[:,i])
            U[i][j] = u_ij
            L[j][i] = l_ji/U[i][i]
    return L, U

#front substitution of lower triangular matrix
def LU_solve(L, U, b, n): #based on algorithm 6.1
    #lower triangular matrix L, upper triangular matrix U, constants matrix b
    y = np.zeros((n, 1))
    for i in range(n): #front solve Ly = b for y
        y_i = b[i]
        for j in range(i):
            y_i = y_i - L[i][j] * y[j]
        y[i] = y_i/L[i][i]
    return Backsolve(U, y, n) #back solve Ux = y for x

epsilon = 1e-8 #tolerance, checked against absolute update

#power method
def power(A, x_0, n): #based on algorithm 9.1
    #matrix A, initial guess x_0
    x = x_0/np.linalg.norm(x_0, np.inf)
    i = 0
    lamda = 1
    while True:
        i += 1
        x_t = x.copy()
        x_t = np.dot(A, x_t)
        lamda = np.linalg.norm(x_t, np.inf)
        x_t = x_t/np.linalg.norm(x_t, np.inf)
        if np.abs(np.linalg.norm(x - x_t)) <= epsilon:
            x = x_t.copy()
            break
        x = x_t.copy()
    return lamda, x

#inverse power method
def inverse_power(A, x_0, n): #based on algorithm 9.3
    #matrix A, initial guess x_0
    x = x_0/np.linalg.norm(x_0, np.inf)
    L, U = LU_factorize(A, n)
    i = 0
    lamda = 1
    while True:
        i += 1
        x_t = x.copy()
        x_t = LU_solve(L, U, x_t, n) #solve using LU factorization
        lamda = np.linalg.norm(x_t, np.inf)
        x_t = x_t/np.linalg.norm(x_t, np.inf)
        if np.abs(np.linalg.norm(x - x_t)) <= epsilon:
            x = x_t.copy()
            break
        x = x_t.copy()
    return 1/lamda, x

def deflation_max(A, lamda, v, x_0, n): #based on algorithm 9.4
    #matrix A, max eigenvalue lamda_max, eigenvector v_max, initial guess x_0
    i = np.argmax(np.abs(v))
    B = np.zeros((n - 1, n - 1))
    for k in range(i):
        for j in range(i):
            B[k][j] = A[k][j] - v[k]/v[i]*A[i][j]
    for k in range(i, n - 1):
        for j in range(i):
            B[k][j] = A[k + 1][j] - v[k + 1]/v[i]*A[i][j]
            B[j][k] = A[j][k + 1] - v[j]/v[i]*A[i][k + 1]
    for k in range(i, n - 1):
        for j in range(i, n - 1):
            B[k][j] = A[k + 1][j + 1] - v[k + 1]/v[i]*A[i][j + 1]
    mu, w1 = power(B, x_0[:-1], n - 1)
    w = np.insert(w1, i, 0)
    u = np.zeros((n, 1))
    for k in range(n):
        u[k] = (mu - lamda) * w[k] + np.dot(A[i], w) * v[k]/v[i]
    return mu, u
def deflation_min(A, lamda, v, x_0, n): #based on algorithm 9.4
    #matrix A, max eigenvalue lamda_max, eigenvector v_max, initial guess x_0
    i = np.argmax(np.abs(v))
    B = np.zeros((n - 1, n - 1))
    for k in range(i):
        for j in range(i):
            B[k][j] = A[k][j] - v[k]/v[i]*A[i][j]
    for k in range(i, n - 1):
        for j in range(i):
            B[k][j] = A[k + 1][j] - v[k + 1]/v[i]*A[i][j]
            B[j][k] = A[j][k + 1] - v[j]/v[i]*A[i][k + 1]
    for k in range(i, n - 1):
        for j in range(i, n - 1):
            B[k][j] = A[k + 1][j + 1] - v[k + 1]/v[i]*A[i][j + 1]
    mu, w1 = power(B, x_0[:-1], n - 1)
    w = np.insert(w1, i, 0)
    u = np.zeros((n, 1))
    for k in range(n):
        u[k] = (mu - lamda) * w[k] + np.dot(A[i], w) * v[k]/v[i]
    if n > 3:
        mu, w1 = deflation_min(B, mu, w1, x_0[:-1], n - 1)
        w = np.insert(w1, i, 0)
        u = np.zeros((n, 1))
        for k in range(n):
            u[k] = (mu - lamda) * w[k] + np.dot(A[i], w) * v[k]/v[i]
        return mu, u
    else:
        return mu, u
        
test = 2
if test == 1:
    n = 2
    A = matrix_init(n, test)
    x_0 = np.ones((n, 1))
    
    eigs, v = np.linalg.eig(A)
    eigs = np.sort(eigs)
    print("two lowest: ", eigs[0], eigs[1])
    print("two highest: ", eigs[-1], eigs[-2])
    
    eig_max, v_max = power(A, x_0, n)
    print(eig_max, v_max)
    print(np.abs(eig_max - eigs[-1]), np.linalg.norm(np.dot(A, v_max) - np.dot(eig_max, v_max), np.inf)/np.abs(eigs[-1]))
    eig_min, v_min = inverse_power(A, x_0, n)
    print(eig_min, v_min)
    print(np.abs(eig_min - eigs[0]), np.linalg.norm(np.dot(A, v_min) - np.dot(eig_min, v_min), np.inf)/np.abs(eigs[0]))
if test == 2:
    n = 50
    time_max = np.array([]) #computation time for max eigenvalue
    lamda_err_max = np.array([]) #eigenvalue error for max eigenvalue
    vec_err_max = np.array([]) #eigenvector error for max eigenvalue
    time_min = np.array([]) #computation time for min eigenvalue
    lamda_err_min = np.array([]) #eigenvalue error for min eigenvalue
    vec_err_min = np.array([]) #eigenvector error for min eigenvalue
    time_max2 = np.array([]) #computation time for 2nd max eigenvalue
    lamda_err_max2 = np.array([]) #eigenvalue error for 2nd max eigenvalue
    vec_err_max2 = np.array([]) #eigenvector error for 2nd max eigenvalue
    time_min2 = np.array([]) #computation time for min 2nd eigenvalue
    lamda_err_min2 = np.array([]) #eigenvalue error for 2nd min eigenvalue
    vec_err_min2 = np.array([]) #eigenvector error for 2nd min eigenvalue
    
    for i in range(25):
        A = matrix_init(n, test)
        x_0 = np.ones((n, 1))

        eigs, v = np.linalg.eig(A)
        eigs = np.sort(eigs)
        #print("two lowest: ", eigs[0], eigs[1])
        #print("two highest: ", eigs[-1], eigs[-2])
        
        t0_max = time.time()
        eig_max, v_max = power(A, x_0, n)
        t1_max = time.time()
        time_max = np.append(time_max, t1_max - t0_max)
        lamda_err_max = np.append(lamda_err_max, np.abs(eig_max - eigs[-1]))
        vec_err_max = np.append(vec_err_max, np.linalg.norm(np.dot(A, v_max) - np.dot(eig_max, v_max), np.inf)/np.abs(eigs[-1]))
        
        t0_min = time.time()
        eig_min, v_min = inverse_power(A, x_0, n)
        t1_min = time.time()
        time_min = np.append(time_min, t1_min - t0_min)
        lamda_err_min = np.append(lamda_err_min, np.abs(eig_min - eigs[0]))
        vec_err_min = np.append(vec_err_min, np.linalg.norm(np.dot(A, v_min) - np.dot(eig_min, v_min), np.inf)/np.abs(eigs[0]))
        
        t0_max2 = time.time()
        eig_max2, v_max2 = deflation_max(A, eig_max, v_max, x_0, n)
        t1_max2 = time.time()
        time_max2 = np.append(time_max2, t1_max2 - t0_max2)
        lamda_err_max2 = np.append(lamda_err_max2, np.abs(eig_max2 - eigs[-2]))
        vec_err_max2 = np.append(vec_err_max2, np.linalg.norm(np.dot(A, v_max2) - np.dot(eig_max2, v_max2), np.inf)/np.abs(eigs[-2]))
        
        t0_min2 = time.time()
        eig_min2, v_min2 = deflation_min(A, eig_max, v_max, x_0, n)
        t1_min2 = time.time()
        time_min2 = np.append(time_min2, t1_min2 - t0_min2)
        lamda_err_min2 = np.append(lamda_err_min2, np.abs(eig_min2 - eigs[1]))
        vec_err_min2 = np.append(vec_err_min2, np.linalg.norm(np.dot(A, v_min2) - np.dot(eig_min2, v_min2), np.inf)/np.abs(eigs[1]))
        print(i) #progress tracker
    
    
    print(np.average(time_max), "\t", np.average(lamda_err_max), "\t", np.average(vec_err_max))
    print(np.average(time_min), "\t", np.average(lamda_err_min), "\t", np.average(vec_err_min))
    print(np.average(time_max2), "\t", np.average(lamda_err_max2), "\t", np.average(vec_err_max2))
    print(np.average(time_min2), "\t", np.average(lamda_err_min2), "\t", np.average(vec_err_min2))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0.022877769470214845 	 1.866343893652811e-07 	 3.1581661648546783e-09
90.20300110816956 	 3.5203752773327324e-07 	 2.8822128973154576e-07
0.05610614776611328 	 2.682591991742811e-07 	 6.642218563932942e-08
2.6324415588378907 	 4.338336559328582e-08 	 8.773889635891821e+58
